In [ ]:
import pandas as pd
import numpy as np
import functools as ft
import itertools as it

import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from copy import copy

from tools import networkHelpers as nh

%matplotlib inline

In [51]:
from scipy.stats import hypergeom
understudied = pd.read_csv('../data/dark_kinases.csv').symbol.tolist()

def test_understudied(clust, understudied, M):
    N = len(clust)
    
    rv = hypergeom(M, len(understudied), N)
    
    k = sum(map(lambda x: x in understudied, clust))
    
    return rv.pmf(k)

In [58]:
understudied_p_vals = []
louv = copy(louvain_combined_dict['go'].drop('GO Labels', axis=1))
louv.head()

,cluster_super,cluster_sub
names,,
MST1R,3,11
YES1,3,11
TYRO3,3,11
FGR,3,11
SRC,3,11


In [59]:
louv['understudied'] = pd.Series(louv.index.to_list()).apply(lambda x: x in understudied).to_list()

In [60]:
understudied_super_p_vals = {}
for i in sorted(louv.cluster_super.unique()):
    understudied_super_p_vals[i]=test_understudied(louv[louv['cluster_super']==i].index.to_list(), understudied, M=len(louv))
    
understudied_sub_p_vals = {}
for i in sorted(louv.cluster_sub.unique()):
    understudied_sub_p_vals[i]=test_understudied(louv[louv['cluster_sub']==i].index.to_list(), understudied, M=len(louv))

In [64]:
sub_super_dict = louv.groupby('cluster_sub')['cluster_super'].agg('mean').to_dict()
sub_super_dict

{1: 4,
 2: 2,
 3: 7,
 4: 6,
 5: 8,
 6: 6,
 7: 1,
 8: 1,
 9: 2,
 10: 6,
 11: 3,
 12: 8,
 13: 5,
 14: 6,
 15: 5,
 16: 4,
 17: 6,
 18: 6,
 19: 4,
 20: 8,
 21: 1,
 22: 6,
 23: 6,
 24: 2,
 25: 6,
 26: 9}

In [65]:
understudied_super_clusts = []
understudied_sub_clusts = []

for clust, p_val in understudied_super_p_vals.items():
    if(p_val < 0.05):
        understudied_super_clusts += [clust]
        
for clust, p_val in understudied_sub_p_vals.items():
    if(p_val < 0.05):
        understudied_sub_clusts += [clust]
        
print(understudied_super_clusts)
print(list(zip(understudied_sub_clusts, [sub_super_dict[x] for x in understudied_sub_clusts])))

[2, 3, 4, 5, 7]
[(1, 4), (2, 2), (3, 7), (7, 1), (11, 3), (13, 5), (19, 4)]


In [66]:
super_under_map = (louv_cluster_df.groupby('cluster_super')['understudied'].agg(sum)/louv_cluster_df.groupby('cluster_super')['understudied'].agg(len)).to_dict()
sub_under_map = (louv_cluster_df.groupby('cluster_sub')['understudied'].agg(sum)/louv_cluster_df.groupby('cluster_sub')['understudied'].agg(len)).to_dict()
louv_cluster_df['super_prop_under'] = louv_cluster_df.cluster_super.apply(super_under_map.get)
louv_cluster_df['super_under_pval'] = louv_cluster_df.cluster_super.apply(understudied_super_p_vals.get)
louv_cluster_df['sub_prop_under'] = louv_cluster_df.cluster_sub.apply(sub_under_map.get)
louv_cluster_df['sub_under_pval'] = louv_cluster_df.cluster_sub.apply(understudied_sub_p_vals.get)

louv_cluster_df.head()

,cluster_super,cluster_sub,understudied,super_prop_under,super_under_pval,sub_prop_under,sub_under_pval
names,,,,,,,
MST1R,3,11,False,0.12,0.000025,0.12,0.000025
YES1,3,11,False,0.12,0.000025,0.12,0.000025
TYRO3,3,11,False,0.12,0.000025,0.12,0.000025
FGR,3,11,False,0.12,0.000025,0.12,0.000025
SRC,3,11,False,0.12,0.000025,0.12,0.000025


In [76]:
agg_dict = {'understudied':len}
agg_dict.update({x:np.mean for x in ['super_prop_under','super_under_pval','sub_prop_under','sub_under_pval']})

understudied_table = louv_cluster_df.groupby(['cluster_super', 'cluster_sub']).agg(agg_dict)
understudied_table['super_prop_under'] = understudied_table['super_prop_under'].apply(lambda x: np.round(x*100,1))
understudied_table['sub_prop_under'] = understudied_table['sub_prop_under'].apply(lambda x: np.round(x*100,1))


raw_columns = ['Subclust\nSize', 'Superclust\n% Understud', 'Superclust \np-value', 'Subclust\n% Understud', 'Subclust \np-value']
multindex_column_tuples = [tuple(col.split('\n')) for col in raw_columns]

understudied_table.columns = pd.MultiIndex.from_tuples(multindex_column_tuples)

for col in understudied_table.columns:
    understudied_table[col] = understudied_table[col].apply(lambda x: np.round(x, 4))
understudied_table


Subclust  Superclust Superclust     Subclust  \
                              Size % Understud     p-value % Understud   
cluster_super cluster_sub                                                
1             7                 17        22.5      0.0697         0.0   
              8                 19        22.5      0.0697        36.8   
              21                 4        22.5      0.0697        50.0   
2             2                123        43.7      0.0001        43.1   
              9                 14        43.7      0.0001        42.9   
              24                 5        43.7      0.0001        60.0   
3             11                75        12.0      0.0000        12.0   
4             1                 52        17.2      0.0056        13.5   
              16                 3        17.2      0.0056         0.0   
              19                 3        17.2      0.0056       100.0   
5             13                31        15.0      0.0091        12.9   
              15                 9        15.0      0.0091        22.2   
6             4                  9        25.0      0.0624        22.2   
              6                  8        25.0      0.0624        12.5   
              10                 3        25.0      0.0624         0.0   
              14                12        25.0      0.0624        33.3   
              17                15        25.0      0.0624        33.3   
              18                 9        25.0      0.0624        44.4   
              22                 2        25.0      0.0624         0.0   
              23                 4        25.0      0.0624         0.0   
              25                 2        25.0      0.0624         0.0   
7             3                 63        14.3      0.0007        14.3   
8             5                 21        28.6      0.1617        19.0   
              12                 5        28.6      0.1617        40.0   
              20                 2        28.6      0.1617       100.0   
9             26                 7        14.3      0.2320        14.3   

                          Subclust   
                            p-value  
cluster_super cluster_sub            
1             7              0.0016  
              8              0.1651  
              21             0.2766  
2             2              0.0005  
              9              0.1390  
              24             0.1429  
3             11             0.0000  
4             1              0.0013  
              16             0.3256  
              19             0.0298  
5             13             0.0109  
              15             0.2574  
6             4              0.2574  
              6              0.1820  
              10             0.3256  
              14             0.2381  
              17             0.2139  
              18             0.1842  
              22             0.4737  
              23             0.2236  
              25             0.4737  
7             3              0.0007  
8             5              0.0976  
              12             0.3181  
              20             0.0966  
9             26             0.2320

In [78]:
understudied_filled_table = copy(understudied_table)
to_keep = np.zeros_like(understudied_filled_table.index.get_level_values(0))

understudied_filled_table = understudied_filled_table[understudied_filled_table.columns[[1,2,0,3,4]]]

for sup_ in understudied_filled_table.index.get_level_values(0):
    local_tab = understudied_filled_table.loc[sup_]
    first_sub = local_tab.index.to_list()[0]
    to_keep = np.logical_or(to_keep, [x == first_sub for x in understudied_filled_table.index.get_level_values(1).to_list()])
    
understudied_filled_table[understudied_filled_table.columns[0]][~to_keep] = ''
understudied_filled_table[understudied_filled_table.columns[1]][~to_keep] = ''

display(understudied_filled_table)
print(understudied_filled_table.to_latex())

Superclust Superclust  Subclust              \
                          % Understud     p-value     Size % Understud   
cluster_super cluster_sub                                                
1             7                  22.5      0.0697       17         0.0   
              8                                         19        36.8   
              21                                         4        50.0   
2             2                  43.7      0.0001      123        43.1   
              9                                         14        42.9   
              24                                         5        60.0   
3             11                   12           0       75        12.0   
4             1                  17.2      0.0056       52        13.5   
              16                                         3         0.0   
              19                                         3       100.0   
5             13                   15      0.0091       31        12.9   
              15                                         9        22.2   
6             4                    25      0.0624        9        22.2   
              6                                          8        12.5   
              10                                         3         0.0   
              14                                        12        33.3   
              17                                        15        33.3   
              18                                         9        44.4   
              22                                         2         0.0   
              23                                         4         0.0   
              25                                         2         0.0   
7             3                  14.3      0.0007       63        14.3   
8             5                  28.6      0.1617       21        19.0   
              12                                         5        40.0   
              20                                         2       100.0   
9             26                 14.3       0.232        7        14.3   

                          Subclust   
                            p-value  
cluster_super cluster_sub            
1             7              0.0016  
              8              0.1651  
              21             0.2766  
2             2              0.0005  
              9              0.1390  
              24             0.1429  
3             11             0.0000  
4             1              0.0013  
              16             0.3256  
              19             0.0298  
5             13             0.0109  
              15             0.2574  
6             4              0.2574  
              6              0.1820  
              10             0.3256  
              14             0.2381  
              17             0.2139  
              18             0.1842  
              22             0.4737  
              23             0.2236  
              25             0.4737  
7             3              0.0007  
8             5              0.0976  
              12             0.3181  
              20             0.0966  
9             26             0.2320

\begin{tabular}{llllrrr}
\toprule
  &    &  Superclust & Superclust  & \multicolumn{2}{l}{Subclust} & Subclust  \\
  &    & \% Understud &     p-value &     Size & \% Understud &   p-value \\
cluster\_super & cluster\_sub &             &             &          &             &           \\
\midrule
1 & 7  &        22.5 &      0.0697 &       17 &         0.0 &    0.0016 \\
  & 8  &             &             &       19 &        36.8 &    0.1651 \\
  & 21 &             &             &        4 &        50.0 &    0.2766 \\
2 & 2  &        43.7 &      0.0001 &      123 &        43.1 &    0.0005 \\
  & 9  &             &             &       14 &        42.9 &    0.1390 \\
  & 24 &             &             &        5 &        60.0 &    0.1429 \\
3 & 11 &          12 &           0 &       75 &        12.0 &    0.0000 \\
4 & 1  &        17.2 &      0.0056 &       52 &        13.5 &    0.0013 \\
  & 16 &             &             &        3 &         0.0 &    0.3256 \\
  & 19 &             &    